xLAM
===

**xLAM: A Family of Large Action Models to Empower AI Agent Systems**

* Paper: https://arxiv.org/abs/2409.03215

![xLAM Overview](../assets/xlam_overview.png)

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "Salesforce/xLAM-2-1b-fc-r"
)
model = AutoModelForCausalLM.from_pretrained(
    "Salesforce/xLAM-2-1b-fc-r",
    torch_dtype=torch.bfloat16, device_map="auto"
)


/home/pyml/anaconda3/envs/birefnet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# define the list of tools that the model can use
tools = [
    {
        "name": "get_weather",
        "description": "Get the current weather for a location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string", "description": "The city and state, e.g. San Francisco, CA"},
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"], "description": "The unit of temperature to return"}
            },
            "required": ["location"]
        }
    },
    {
        "name": "get_directions",
        "description": "Get directions from one location to another",
        "parameters": {
            "type": "object",
            "properties": {
                "location1": {"type": "string", "description": "The starting location"},
                "location2": {"type": "string", "description": "The destination location"},
                "mode": {"type": "string", "enum": ["driving", "walking", "bicycling"], "description": "The mode of transportation"}
            },
            "required": ["location1", "location2"]
        }
    }
]

In [6]:
# Example conversation with a tool call
messages = [
    {"role": "user", "content": "Hi, how are you?"},
    {"role": "assistant", "content": "Thanks. I am doing well. How can I help you?"},
    {"role": "user", "content": "Can you tell me the weather in San Francisco, CA?"},
]

print("====== prompt after applying chat template ======")
print(tokenizer.apply_chat_template(messages, tools=tools, add_generation_prompt=True, tokenize=False))


====== prompt after applying chat template ======
<|im_start|>system
You are a helpful assistant that can use tools. You are developed by Salesforce xLAM team.
You have access to a set of tools. When using tools, make calls in a single JSON array: 

[{"name": "tool_call_name", "arguments": {"arg1": "value1", "arg2": "value2"}}, ... (additional parallel tool calls as needed)]

If no tool is suitable, state that explicitly. If the user's input lacks required parameters, ask for clarification. Do not interpret or respond until tool results are returned. Once they are available, process them or make additional calls if needed. For tasks that don't require tools, such as casual conversation or general advice, respond directly in plain text. The available tools are:

{
    "name": "get_weather",
    "description": "Get the current weather for a location",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
        

In [8]:

def run(messages, tools):
    inputs = tokenizer.apply_chat_template(
        messages,
        tools=tools,
        add_generation_prompt=True,
        return_dict=True,
        return_tensors="pt"
    )

    # Get the length of the input tokens
    input_ids_len = inputs["input_ids"].shape[-1]
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    print("====== model response ======")
    outputs = model.generate(**inputs, max_new_tokens=256)
    generated_tokens = outputs[:, input_ids_len:] # Slice the output to get only the newly generated tokens
    print(tokenizer.decode(generated_tokens[0], skip_special_tokens=True))

run(messages, tools)

====== model response ======
[{"name": "get_weather", "arguments": {"location": "San Francisco, CA"}}]


In [9]:
# example 2: directions
messages = [
    {"role": "user", "content": "Hi, how are you?"},
    {"role": "assistant", "content": "Thanks. I am doing well. How can I help you?"},
    {"role": "user", "content": "Can you give me directions from San Francisco, CA to Los Angeles, CA?"},
]

run(messages, tools)

====== model response ======
[{"name": "get_directions", "arguments": {"location1": "San Francisco, CA", "location2": "Los Angeles, CA"}}]
